In [1]:
import pandas as pd

## Wczytanie danych bets

In [86]:
for year in range(2018, 2021):
    file_name=f"Raw data/{year}.xlsx"
    variable_name=f"bets_{year}"
    globals()[variable_name] = pd.read_excel(file_name)
    print(f"Loaded {variable_name} from {file_name}")

Loaded bets_2018 from Raw data/2018.xlsx
Loaded bets_2019 from Raw data/2019.xlsx
Loaded bets_2020 from Raw data/2020.xlsx


In [87]:
for i in range(2018,2021):
    variable_name=f"bets_{i}"
    globals()[variable_name].loc[globals()[variable_name]["Tournament"] == "Adelaide International 1", "Location"] = "Adelaide 1"
    globals()[variable_name].loc[globals()[variable_name]["Tournament"] == "Adelaide International 2", "Location"] = "Adelaide 2"
    globals()[variable_name].loc[globals()[variable_name]["Tournament"] == "Western & Southern Financial Group Masters", "Location"] = "Cincinnati"
    globals()[variable_name].loc[globals()[variable_name]["Tournament"] == "bett1HULKS Indoors", "Location"] = "Cologne 1"
    globals()[variable_name].loc[globals()[variable_name]["Tournament"] == "bett1HULKS Championship", "Location"] = "Cologne 2"
    globals()[variable_name]["Location"] = globals()[variable_name]["Location"].replace({'Dubai ': 'Dubai', 'Belgrade ': 'Belgrade', 'Napoli':'Naples', 'Montreal': 'Toronto', 'Shenzhen ': 'Shenzhen', 'St Petersburg': 'St. Petersburg'})
    globals()[variable_name] = globals()[variable_name][~globals()[variable_name]['Location'].isin(['Turin'])]
    globals()[variable_name]['Loser'] = globals()[variable_name]['Loser'].replace({"Varillas J. P.": "Varillas J.P.", "Tseng C. H.": "Tseng C.H."})
    globals()[variable_name]['Winner'] = globals()[variable_name]['Winner'].replace({"Varillas J. P.": "Varillas J.P.", "Tseng C. H.": "Tseng C.H."})
    globals()[variable_name] = globals()[variable_name][~globals()[variable_name]['Tournament'].isin(['United Cup', 'Tour Finals', 'NextGen Finals','Tokyo Olympics','Atp Cup', 'Great Ocean Road Open', 'Murray River Open',
     'Laver Cup', 'Melbourne Summer Set'])]
    globals()[variable_name].loc[globals()[variable_name]['Tournament']== 'BNP Paribas Masters', 'Location']='Paris 2'
    globals()[variable_name].loc[globals()[variable_name]['Tournament']== 'Belgrade Open', 'Location']='Belgrade 2'

    
    
    
    

### W ramce danych z 2019 roku jest błędnie wpisany zawodnik wraz z rankingiem. Poprawiamy ręcznie błąd

In [88]:
bets_2019.loc[bets_2019['Loser'] == 'Monteiro J.', 'LRank'] = 99
bets_2019.loc[bets_2019['Loser'] == 'Monteiro J.', 'LPts'] = 580.0
bets_2019.loc[bets_2019['Loser'] == 'Monteiro J.', 'Loser'] = 'Monteiro T.'

### W 2020 roku w ramce bets nazwa 2 turniejów jest ze sobą błednie zamieniona. Poprawiamy ten błąd zamieniając nazwy turniejów na poprawne.

In [90]:
auckland_mask = (bets_2020['Location'] == 'Auckland') & (bets_2020['Tournament'] == 'ASB Classic')
adelaide_mask = (bets_2020['Location'] == 'Adelaide') & (bets_2020['Tournament'] == 'Adelaide International')

bets_2020.loc[auckland_mask, ['Location', 'Tournament']] = ['Adelaide', 'Adelaide International']
bets_2020.loc[adelaide_mask, ['Location', 'Tournament']] = ['Auckland', 'ASB Classic']

In [91]:
for i in range(2018,2021):
    variable_name=f"bets_{i}"
    globals()[variable_name].rename(columns={'Location': 'tourney_location'}, inplace=True)
    globals()[variable_name].reset_index(drop=True, inplace=True)

## ZBIORY RÓŻNIĄ SIĘ BUKMACHERAMI, DLATEGO ZBIERAMY WSZYSTKIE UNIKATOWE NAZWY BUKMACHEROW I UPEWNIAMY SIE, ŻE KAŻDY ZBIÓR MA KAŻDEGO BUKMACHERA POPRZEZ DODANIE KOLUMNY, WARTOŚCI DLA NOWO DODANEJ KOLUMNY SĄ NAN

In [92]:
for i in range(2018, 2021):
    variable_name = f"bets_{i}"
    variable_name_bookmakers = f"bets_{i}_bookmakers"
    globals()[variable_name_bookmakers] = globals()[variable_name].copy()
    
    columns_to_drop = ['ATP', 'tourney_location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
                       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1',
                       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
                       'Comment']
    
    globals()[variable_name_bookmakers] = globals()[variable_name_bookmakers].drop(columns=columns_to_drop)
    

## ZEBRANIE WSZYSTKICH UNIKALNYCH NAZW BUKMACHERÓW

In [93]:
unique_columns = set()
for i in range(2018, 2021):
    variable_name_bookmakers = f"bets_{i}_bookmakers"
    if variable_name_bookmakers in globals() and isinstance(globals()[variable_name_bookmakers], pd.DataFrame):
        unique_columns.update(globals()[variable_name_bookmakers].columns)


## UZUPEŁNIENIE BRAKUJĄCYCH KOLUMN W bets_{rok} I USTAWIENIE WARTOŚCI NA NAN

In [94]:
for i in range(2018, 2021):
    variable_name = f"bets_{i}"
    if variable_name in globals() and isinstance(globals()[variable_name], pd.DataFrame):
        missing_columns = unique_columns.difference(globals()[variable_name].columns)
        for column in missing_columns:
            globals()[variable_name][column] = None

## PONIŻSZA FUNKCJA NIE MA ŻADNEGO OUTPUTU, TZN ŻE WSZYSTKO JEST W PORZĄDKU I DANE MOŻNA ZMERGOWAĆ

In [95]:
for i in range(2018, 2021):
    variable_name=f"bets_{i}"
    columns=set(globals()[variable_name].columns)
    for j in range(i+1, 2021):
        variable_name_2=f"bets_{j}"
        if columns != set(globals()[variable_name_2].columns):
            print(f"Rok: {i} niezgodny z rokiem {j}")

## Wczytanie danych github

In [96]:
for i in range(2018, 2021):
    variable_name=f"github_{i}"
    file_name = f"Raw data/atp_matches_{i}.csv"
    globals()[variable_name] = pd.read_csv(file_name)
    print(f"Loaded {variable_name} from {file_name}")

Loaded github_2018 from Raw data/atp_matches_2018.csv
Loaded github_2019 from Raw data/atp_matches_2019.csv
Loaded github_2020 from Raw data/atp_matches_2020.csv


## wszystkie zbiory mają jednakowe kolumny :))

In [97]:
k=0
for i in range(2018, 2021):
    variable_name=f"github_{i}"
    columns=set(globals()[variable_name].columns)
    for j in range(i+1, 2021):
        variable_name_2=f"github_{j}"
        if columns != set(globals()[variable_name_2].columns):
            print(f"Rok: {i} niezgodny z rokiem {j}")
            k+=1
print(k)

0


### Zgodnie z założeniami usuwamy Finals, NextGen Finals, United Cup oraz mecze Davis Cup

In [98]:
for i in range(2018, 2021):
    variable_name=f"github_{i}"
    globals()[variable_name]=globals()[variable_name][~globals()[variable_name]['tourney_name'].str.contains('Davis Cup', na=False)]
    globals()[variable_name] = globals()[variable_name][~globals()[variable_name]['tourney_name'].isin(['United Cup', 'Tour Finals', 'NextGen Finals','Tokyo Olympics','Atp Cup', 'Great Ocean Road Open', 'Murray River Open',
  'Laver Cup','Melbourne Summer Set', 'Melbourne'])]
    globals()[variable_name]["tourney_name"] = globals()[variable_name]["tourney_name"].replace({'Belgrade ': 'Belgrade'})
    
    

### usunięcie turnieju, który występuje wyłącznie w jednej z ramek. Uniemożliwia to walidację, wolimy turniej usunąć i mieć pewniejsze dane niż zaufać jednemu ze źródeł

In [99]:
bets_2020=bets_2020[~bets_2020['tourney_location'].str.contains('London', na=False)]
bets_2018 = bets_2018[~((bets_2018['tourney_location'].str.contains('London', na=False)) & (bets_2018['Series'] == 'Masters Cup'))]
bets_2019 = bets_2019[~((bets_2019['tourney_location'].str.contains('London', na=False)) & (bets_2019['Series'] == 'Masters Cup'))]

### W tej ramce kolumna 'tourney_name' to dla większości turniejów lokalizacja turnieju a nie jego nazwa, a potrzebujemy żeby nazwa turnieju była taka jak w ramce bets. Dla turniejów, dla których 'tourney_name' to nie lokalizacja, ręcznie wpisujemy lokalizację

In [100]:
for i in range(2018, 2021):
    variable_name=f"github_{i}"
    globals()[variable_name].rename(columns={'tourney_name': 'tourney_location'}, inplace=True)
    globals()[variable_name]['tourney_location'] = globals()[variable_name]['tourney_location'].replace({"Australian Open": "Melbourne", "Indian Wells Masters": "Indian Wells", "Miami Masters": "Miami", "Monte Carlo Masters": "Monte Carlo", "Madrid Masters": "Madrid", "Rome Masters": "Rome", "Roland Garros": "Paris", "s Hertogenbosch": "'s-Hertogenbosch", "Queen's Club": "Queens Club", "Wimbledon": "London", "Canada Masters": "Toronto", "Cincinnati Masters": "Cincinnati", "Us Open": "New York", "US Open": "New York", "Astana": "Nur-Sultan", "Shanghai Masters": "Shanghai", "Paris Masters": "Paris 2", "Rio De Janeiro": "Rio de Janeiro", "Shenzen ":"Shenzen", "ATP Rio de Janeiro": "Rio de Janeiro",'St Petersburg': 'St. Petersburg'})

### Sprawdzamy, że utworzona przez nas kolumnna 'tourney_location' odpowiada kolumnie 'tourney_location' w ramce bets

In [101]:
for i in range(2018, 2021):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    print(globals()[variable_name][~globals()[variable_name]['tourney_location'].isin(globals()[variable_name_2]['tourney_location'])]['tourney_location'].unique())
    print(globals()[variable_name_2][~globals()[variable_name_2]['tourney_location'].isin(globals()[variable_name]['tourney_location'])]['tourney_location'].unique())
    
    

[]
[]
[]
[]
[]
[]


### Dzięki temu możemy zamienić wartości kolumny 'tourney_level' na wartości kolumny 'Series' z ramki bets. Dziwne określenie rangi turnieju za pomoca literek zmieniamy na standardowe rangi ATP250, ATP500 etc. Ręcznie musimy jedynie poprawić turniej Masters 1000 w Paryżu, gdyż również w Paryżu odbywa się turniej rangi Grand Slam.

In [102]:
for i in range(2018, 2021):
    github_name = f"github_{i}"
    bets_name = f"bets_{i}"
    
    github_df = globals()[github_name]
    bets_df = globals()[bets_name]
    df2=bets_df.drop_duplicates(subset='tourney_location', keep='first')

    merged_df = pd.merge(github_df, df2[['tourney_location', 'Series']], on='tourney_location', how='left')
    merged_df['tourney_level'] = merged_df['Series']
    merged_df = merged_df.drop(columns=['Series'])
    globals()[github_name]=merged_df
    

In [103]:
for i in range(2018,2021):
    variable_name=f"github_{i}"
    globals()[variable_name].loc[(globals()[variable_name]["draw_size"] == 128) & (globals()[variable_name]["tourney_location"] == "New York"), "tourney_level"] = "Grand Slam"

### Możemy teraz stworzyć w obu ramkach klucz 'tournament_id', który jest połaczeniem 'tourney_location' oraz 'tourney_level' i jednoznacznie określa turniej

In [104]:
for i in range(2018,2021):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    globals()[variable_name]['tournament_id']=globals()[variable_name]['tourney_location'].astype(str)+ '_' + globals()[variable_name]['tourney_level'].astype(str)+ '_' + str(i)
    globals()[variable_name_2]['tournament_id']=globals()[variable_name_2]['tourney_location'].astype(str)+ '_' + globals()[variable_name_2]['Series'].astype(str)+ '_' + str(i)
    

C:\Users\kacpe\AppData\Local\Temp\ipykernel_25176\45324509.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[variable_name_2]['tournament_id']=globals()[variable_name_2]['tourney_location'].astype(str)+ '_' + globals()[variable_name_2]['Series'].astype(str)+ '_' + str(i)


### Sprawdzamy czy w obu ramkach mamy te same klucze turnieju

In [105]:
for i in range(2018, 2021):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    print(globals()[variable_name][~globals()[variable_name]['tournament_id'].isin(globals()[variable_name_2]['tournament_id'])]['tournament_id'].unique())
    print(globals()[variable_name_2][~globals()[variable_name_2]['tournament_id'].isin(globals()[variable_name]['tournament_id'])]['tournament_id'].unique())

[]
[]
[]
[]
[]
[]


### Potrzebujemy mieć takie same nazwy zawodników w obu ramkach, aby dodać do ramki bets id zawodnika z ramki github. Aktualnie w ramce bets nazwa zawodnika jest w formacie typu Djokovic N. a w ramce github w formacie typu Novak Djokovic, dlatego musimy stworzyć nazwy zawodników w skróconym w formacie w ramce github.

In [106]:
def transform_name(name):
    name_parts = name.split()
    first_name = name_parts[0]
    last_name = ' '.join(name_parts[1:])
    return f"{last_name} {first_name[0]}."

In [107]:
for i in range(2018, 2021):
    variable_name=f"github_{i}"
    globals()[variable_name]['shortened_winner_name']=globals()[variable_name]['winner_name'].apply(transform_name)
    globals()[variable_name]['shortened_loser_name']=globals()[variable_name]['loser_name'].apply(transform_name)

### Sprawdzamy czy skrócona nazwa zawodnika jednoznacznie określa zawodnika

In [111]:
for i in range(2018, 2021):
    variable_name=f"github_{i}"
    print(globals()[variable_name]['shortened_winner_name'].nunique() == globals()[variable_name]['winner_id'].nunique())
    print(globals()[variable_name]['shortened_loser_name'].nunique() == globals()[variable_name]['loser_id'].nunique())

True
True
True
True
True
True


## Zobaczmy, którzy zawodnicy nie są jednoznacznie określeni przez skróconą nazwę

In [109]:
non_unique_losers={}
for i in range(2018,2021):
    variable_name=f"github_{i}"
    loser_groups = globals()[variable_name].groupby('shortened_loser_name')['loser_id'].nunique()
    non_unique_loser_names = loser_groups[loser_groups > 1].index.tolist()
    non_unique_losers[i] = globals()[variable_name][globals()[variable_name]['shortened_loser_name'].isin(non_unique_loser_names)][['shortened_loser_name', 'loser_id']].drop_duplicates()

for i in range(2018,2021):
    if not non_unique_losers[i].empty:
        print(f"Lata {i} - Przegrani z niejednoznacznymi skróconymi nazwami:")
        print(non_unique_losers[i])

Lata 2018 - Przegrani z niejednoznacznymi skróconymi nazwami:
     shortened_loser_name  loser_id
2299             Zhang Z.    111190
2400             Zhang Z.    105585
Lata 2019 - Przegrani z niejednoznacznymi skróconymi nazwami:
     shortened_loser_name  loser_id
2274             Zhang Z.    111190
2276             Zhang Z.    105585


## W celu rozróżnienia zawodników do skrótu imienia jednego z nich dopsujemy drugą literkę imienia

In [110]:
for i in range(2018, 2021):
   variable_name=f"github_{i}"
   globals()[variable_name].loc[globals()[variable_name]['loser_id'] == 111190, 'shortened_loser_name'] = 'Zhang Zh.'
   globals()[variable_name].loc[globals()[variable_name]['winner_id'] == 111190, 'shortened_winner_name'] = 'Zhang Zh.'
   globals()[variable_name].loc[globals()[variable_name]['loser_id'] == 105585, 'shortened_loser_name'] = 'Zhang Ze.'
   globals()[variable_name].loc[globals()[variable_name]['winner_id'] == 105585, 'shortened_winner_name'] = 'Zhang Ze.'


### Niektóre imiona i nazwiska ze względu na swoją unikalność lub brak konsekwencji w zapisie musimy zmodyfikować ręcznie

In [112]:
def replace_shortened_names(dataframes):
    replacements = {
        "Varillas J.P.": "Varillas J. P.",
        "Meligeni Alves F.": "Meligeni Rodrigues F",
        "Arnaud Bailly G.": "Bailly G.",
        "Sung Nam J.": "Nam J.S.",
        "Chan Hong S.": "Hong S.",
        "Fa Rodriguez Taverna S.": "Rodriguez Taverna S.",
        "Pucinelli De Almeida M.": "Pucinelli de Almeida M.",
        "Alejandro Hernandez Serrano J.": "Hernandez A.",
        "Marcel Stebe C.": "Stebe C.M.",
        "Martin del Potro J.": "Del Potro J.M.",
        "Marco Moroni G.": "Moroni G.M.",
        "Tsonga J.": "Tsonga J.W.",
        "Ignacio Londero J.": "Londero J.I.",
        "Pablo Varillas J.": "Varillas J. P.",
        "Pablo Ficovich J.": "Ficovich J.P.",
        "C.H. Tseng": "Tseng C. H.",
        "Oconnell C.": "O Connell C.",
        "Elahi Galan D.": "Galan D.E.",
        "Auger Aliassime F.": "Auger-Aliassime F.",
        "Woo Kwon S.": "Kwon S.W.",
        "Barrios Vera T.": "Barrios M.",
        "Yunchaokete B.": "Bu Y.",
        "Manuel Cerundolo J.": "Cerundolo J.M.",
        "Martin Etcheverry T.": "Etcheverry T.",
        "Hugues Herbert P.": "Herbert P.H.",
        "Hsiou Hsu Y.": "Hsu Y.",
        "Andrea Huesler M.": "Huesler M.A.",
        "Kuznetsov A.": "Kuznetsov An.",
        "Son Kwiatkowski T.": "Kwiatkowski T.S.",
        "Li Z.": "Li Zh.",
        "Hsin Tseng C.": "Tseng C. H.",
        "Kumar Mukund S.": "Mukund S.",
        "Ramos A.": "Ramos-Vinolas A.",
        "J Wolf J.": "Wolf J.J.",
        "Zhang Ze": "Zhang Ze.",
        "Lin Wu T.": "Wu T.L.",
        "Hans Rehberg M.": "Rehberg M.",
        "Mpetshi Perricard G.": "Mpetshi G.",
        "Agustin Tirante T.": "Tirante T.A.",
        "Alberto Olivieri G.": "Olivieri G.",
        "Nicolae Madaras D.": "Madaras D.",
        "Cong Mo Y.": "Mo Y.",
        "Patrick Smith J.": "Smith J.P.",
        "Hsun Lu Y.":"Lu Y.H.",
        "Barrios M.":"Barrios Vera M.T.",
        "Lennard Struff J.":"Struff J.L.",
        "Shannan Zayid M.":"Zayid M.S.",
        "Garcia Lopez G.":"Garcia-Lopez G.",
        "Rubin Statham J.":"Statham J.",
        "Li Zh.":"Li Z.",
        "Menendez Maceiras A.":"Menendez-Maceiras A.",
        "Mingjie Lin J.":"Lin J.M.",
        "Aragone J.":"Aragone JC",
        "Moroni G.M.":"Moroni G.",
        "Hee Lee D.":"Lee D.H.",
        "Haider Maurer A.":"Haider-Maurer A.",
        "Estrella V.":"Estrella Burgos V.",
        "Ferreira Silva F.":"Silva F.F.",
        "Gimeno Traver D.":"Gimeno-Traver D.",
        "Samper Montana J.":"Samper-Montana J.",
        "Hernandez J.":"Hernandez-Fernandez J"
    }
    
    for df in dataframes:
        for old_name, new_name in replacements.items():
            df['shortened_winner_name'] = df['shortened_winner_name'].replace({old_name: new_name})
            df['shortened_loser_name'] = df['shortened_loser_name'].replace({old_name: new_name})

dataframes = [github_2018, github_2019, github_2020]
replace_shortened_names(dataframes)


In [113]:

def replace_names(dataframes):
    replacements = {
        "Tseng C.H.": "Tseng C. H.",
        "Varillas J.P.": "Varillas J. P.",
        "Meligeni Alves F.": "Meligeni Rodrigues F",
        "Zhang Ze":"Zhang Ze.",
        "O'Connell C.":"O Connell C.",
        "Galan D.": "Galan D.E.",
        "Aragone J.":"Aragone JC",
        "Dolgopolov O.":"Dolgopolov A.",
    }
    
    for df in dataframes:
        for old_name, new_name in replacements.items():
            df['Loser'] = df['Loser'].replace({old_name: new_name})
            df['Winner']= df['Winner'].replace({old_name:new_name})

dataframes = [bets_2018, bets_2019, bets_2020]
replace_names(dataframes)

C:\Users\kacpe\AppData\Local\Temp\ipykernel_25176\2619847961.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Loser'] = df['Loser'].replace({old_name: new_name})
C:\Users\kacpe\AppData\Local\Temp\ipykernel_25176\2619847961.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Winner']= df['Winner'].replace({old_name:new_name})


## Po poprawkach każdy zawodnik w ramce bets ma już id

In [114]:
for i in range(2018,2021):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    map_player_name_player_id = globals()[variable_name].groupby('shortened_loser_name', as_index=False)[['shortened_loser_name', 'loser_id']].first()
    mapping = dict(zip(map_player_name_player_id['shortened_loser_name'], map_player_name_player_id['loser_id']))
    globals()[variable_name_2]['loser_id'] = globals()[variable_name_2]['Loser'].map(mapping)
    

C:\Users\kacpe\AppData\Local\Temp\ipykernel_25176\167549120.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[variable_name_2]['loser_id'] = globals()[variable_name_2]['Loser'].map(mapping)


In [115]:
bets_2020.loc[bets_2020['loser_id'].isna(), 'Loser'].unique()

array([], dtype=object)

In [116]:
bets_2019.loc[bets_2019['loser_id'].isna(), 'Loser'].unique()

array([], dtype=object)

In [117]:
bets_2018.loc[bets_2018['loser_id'].isna(), 'Loser'].unique()

array([], dtype=object)

In [118]:
for i in range(2018,2021):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    map_player_name_player_id = globals()[variable_name].groupby('shortened_winner_name', as_index=False)[['shortened_winner_name', 'winner_id']].first()
    mapping = dict(zip(map_player_name_player_id['shortened_winner_name'], map_player_name_player_id['winner_id']))
    globals()[variable_name_2]['winner_id'] = globals()[variable_name_2]['Winner'].map(mapping)

C:\Users\kacpe\AppData\Local\Temp\ipykernel_25176\135999362.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[variable_name_2]['winner_id'] = globals()[variable_name_2]['Winner'].map(mapping)


In [119]:
bets_2020.loc[bets_2020['winner_id'].isna(), 'Winner'].unique()

array([], dtype=object)

In [120]:
bets_2019.loc[bets_2019['winner_id'].isna(), 'Winner'].unique()

array([], dtype=object)

In [121]:
bets_2018.loc[bets_2018['winner_id'].isna(), 'Winner'].unique()

array([], dtype=object)

### Skoro mamy już wspólne id zawodnika, oraz id turnieju to możemy w obu ramkach zdefiniować wspólne id meczu. Sprawdzamy czy id meczu jest unikalne dla każdego wiersza w ramkach

In [122]:
for i in range(2018, 2021):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    globals()[variable_name]['match_id'] = globals()[variable_name]['tournament_id'].astype(str) + '_' +  globals()[variable_name]['winner_id'].astype(str) + '_' + globals()[variable_name]['loser_id'].astype(str)
    globals()[variable_name_2]['match_id'] = globals()[variable_name_2]['tournament_id'].astype(str) + '_' +  globals()[variable_name_2]['winner_id'].astype(str) + '_' + globals()[variable_name_2]['loser_id'].astype(str)

C:\Users\kacpe\AppData\Local\Temp\ipykernel_25176\4264914879.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[variable_name_2]['match_id'] = globals()[variable_name_2]['tournament_id'].astype(str) + '_' +  globals()[variable_name_2]['winner_id'].astype(str) + '_' + globals()[variable_name_2]['loser_id'].astype(str)


In [123]:
for i in range(2018,2021):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    print(globals()[variable_name]['match_id'].nunique() == len(globals()[variable_name]))
    print(globals()[variable_name_2]['match_id'].nunique() == len(globals()[variable_name_2]))


True
True
True
True
True
True


### Sprawdźmy jeszcze czy sa jakieś mecze, które sa tylko w jednej ramce.

In [124]:
for i in range(2018,2021):
    variable_name=f"github_{i}"
    variable_name_2=f"bets_{i}"
    print(globals()[variable_name_2][~globals()[variable_name_2]['match_id'].isin(globals()[variable_name]['match_id'])]['match_id'].unique())
    print(globals()[variable_name][~globals()[variable_name]['match_id'].isin(globals()[variable_name_2]['match_id'])]['match_id'].unique())
    

[]
[]
[]
[]
[]
[]


In [126]:
folder_path = '../Tennis/Processed data/'
bets_2020.to_csv(folder_path +'bets2020.csv', index=False)
bets_2019.to_csv(folder_path +'bets2019.csv', index=False)
bets_2018.to_csv(folder_path +'bets2018.csv', index=False)
github_2020.to_csv(folder_path +'github2020.csv', index=False)
github_2019.to_csv(folder_path +'github2019.csv', index=False)
github_2018.to_csv(folder_path +'github2018.csv', index=False)